In [ ]:
import sys, os, time

import open3d as o3d
import trimesh
import openmesh as om

import cv2

import scipy as sp
from scipy.sparse.linalg import eigs
from scipy.sparse import csr_matrix, coo_matrix, identity, vstack, diags
from scipy.sparse.linalg import spsolve, lsqr
from scipy.spatial import Delaunay, Voronoi

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib inline

In [ ]:
# Tests of Openmesh
# https://gitlab.vci.rwth-aachen.de:9000/OpenMesh/openmesh-python/tree/master/tests

In [ ]:

def read_mesh_om(path_in_models):
    return om.read_trimesh(os.path.join('..', 'models', path_in_models))
def write_mesh_om(mesh, path_in_models):
    om.write_mesh(os.path.join('..', 'models', path_in_models), mesh, vertex_color=True)
def show_mesh_o3d(plys):
    o3d.visualization.draw_geometries(plys)
def write_mesh_o3d(path, mesh):
    o3d.io.write_triangle_mesh(path, mesh)
def read_mesh_o3d(mesh_fp):
    return o3d.io.read_triangle_mesh(mesh_fp)
def read_mesh_trimesh(path_in_models, process=True):
    return trimesh.load(os.path.join('..', 'models', path_in_models), process=process)
def get_vertices_from_trimesh(mesh):
    return np.asarray(mesh.vertices)
def get_vertice_normals_from_trimesh(mesh):
    return np.asarray(mesh.vertex_normals)
def write_trimesh_with_color(mesh, path, colors):
    o3d_mesh = mesh.as_open3d
    o3d_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    write_mesh_o3d(path, o3d_mesh)

### Calculate Voronoi Poles

In [ ]:
INFINITY = 1e9
EPS = 1e-9
def calculate_voronoi_poles(mesh):
    
    vertices = get_vertices_from_trimesh(mesh)
    vertice_normals = get_vertice_normals_from_trimesh(mesh)

    print("Calculating Voronoi Diagram.")
    vor = Voronoi(vertices)
    vor_centers = vor.vertices
    cells = vor.regions
    cell_indices = vor.point_region

    vertices_total = vertices.shape[0]
    voronoi_poles = np.zeros(vertices.shape)
    print("Calculating the Voronoi Pole for each vertex.")
    for vi in tqdm(range(vertices_total)):
        vor_cell = cells[cell_indices[vi]]
        vertice = vertices[vi]
        vertice_normal = vertice_normals[vi]
        max_neg_proj = INFINITY
        voronoi_pole = None
        for vci in vor_cell:
            vor_center = vor_centers[vci]
            if vci == -1:
                continue
            proj = np.dot(vor_center - vertice, vertice_normal)
            if proj < max_neg_proj:
                max_neg_proj = proj
                voronoi_pole = vor_center
        voronoi_poles[vi] = voronoi_pole

    return voronoi_poles

### Calculate Cotangent Laplace Operator

In [ ]:
def trimesh_Generate_Area_List(mesh):

    area_list = np.zeros((len(mesh.vertices)))
    face_count = mesh.faces.shape[0]
    face_area = np.asarray(mesh.area_faces)
    faces = np.asarray(mesh.faces)
    for i in range(face_count):
        area_list[faces[i]] += face_area[i] / 3
    
    # area_list /= np.mean(area_list)
    print('Mean Area:', np.mean(area_list))
    return area_list


def trimesh_Generate_Laplace_matrix(mesh, is_fixed):
    
    print('Constructing Laplace Matrix.')
    
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_dict = {}
    area_list = trimesh_Generate_Area_List(mesh)
    face_angles = np.asarray(mesh.face_angles)
    faces = np.asarray(mesh.faces)
    face_count = faces.shape[0]
    vertices = np.asarray(mesh.vertices)
    
    with tqdm(total=face_count) as tbar:
        for face, angles in zip(faces, face_angles):
            for i in range(3):
                current_angle = angles[i]
                v0_index, v1_index, v2_index = face[vertices_face_indexs[i]]
                # el = np.linalg.norm(vertices[v1_index] - vertices[v2_index])
                delta = 1 / (np.tan(current_angle) + 1e-8)
                if delta > EPS:
                    if not is_fixed[v1_index]:
                        laplace_dict[(v1_index, v1_index)] = laplace_dict.get((v1_index, v1_index), 0) - delta
                        laplace_dict[(v1_index, v2_index)] = laplace_dict.get((v1_index, v2_index), 0) + delta
                    if not is_fixed[v2_index]:
                        laplace_dict[(v2_index, v2_index)] = laplace_dict.get((v2_index, v2_index), 0) - delta
                        laplace_dict[(v2_index, v1_index)] = laplace_dict.get((v2_index, v1_index), 0) + delta
            tbar.update(1)

    # Construct CSR Matrix
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())
    coo = coo_matrix((values, (rows, cols)), shape=(len(mesh.vertices), len(mesh.vertices)))
    csr = coo.tocsr()

    return csr

def trimesh_Generate_Tangent_Laplace_matrix(mesh, is_fixed, T):
    
    print('Constructing Laplace Matrix.')
    
    vertices_face_indexs = [[0,1,2],[1,0,2],[2,0,1]] 
    laplace_dict = {}
    face_angles = np.asarray(mesh.face_angles)
    faces = np.asarray(mesh.faces)
    face_count = faces.shape[0]
    vertices = np.asarray(mesh.vertices)
    
    with tqdm(total=face_count) as tbar:
        for face, angles in zip(faces, face_angles):
            for i in range(3):
                alpha0, alpha1, alpha2 = angles[vertices_face_indexs[i]]
                v0_index, v1_index, v2_index = face[vertices_face_indexs[i]]
                el = np.linalg.norm(vertices[v1_index] - vertices[v2_index]) + 1e-8
                if not is_fixed[v1_index]:
                    laplace_dict[(v1_index, v1_index)] = laplace_dict.get((v1_index, v1_index), 0) - np.tan(alpha1 / 2.) / el
                    laplace_dict[(v1_index, v2_index)] = laplace_dict.get((v1_index, v2_index), 0) + np.tan(alpha1 / 2.) / el
                if not is_fixed[v2_index]:
                    laplace_dict[(v2_index, v2_index)] = laplace_dict.get((v2_index, v2_index), 0) - np.tan(alpha2 / 2.) / el
                    laplace_dict[(v2_index, v1_index)] = laplace_dict.get((v2_index, v1_index), 0) + np.tan(alpha2 / 2.) / el
            tbar.update(1)

    # Construct CSR Matrix
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())
    coo = coo_matrix((values, (rows, cols)), shape=(len(mesh.vertices), len(mesh.vertices)))
    csr = coo.tocsr()

    return csr

In [ ]:
np.tan(22.5/180*np.pi)

### Solve Equation

In [ ]:
def remove_zero_rows_sparse(sparse_matrix):
    sparse_matrix = csr_matrix(sparse_matrix)
    non_zero_rows = np.diff(sparse_matrix.indptr) > 0
    filtered_matrix = sparse_matrix[non_zero_rows]
    return filtered_matrix

In [ ]:
def construct_Equation(WL, WH, WM, vertices, L, voronoi_poles):
    vn = vertices.shape[0]
    A = vstack((WL * L, WH, WM))
    b = np.vstack((np.zeros((vn, 3)), WH * vertices, WM * voronoi_poles))
    print(A.shape, b.shape, np.sum(A), np.sum(b))
    return A, b

### Collapse Edges

In [ ]:
def get_edges_unique_idx(edges_unique, v1, v2):
    e = np.where((edges_unique[:, 0]==v1)&(edges_unique[:, 1]==v2))
    if e[0].shape[0] == 0:
        e = np.where((edges_unique[:, 0]==v2)&(edges_unique[:, 1]==v1))
    return e[0].item()
a = np.array([
    [1, 2],
    [2, 3],
    [1, 6],
    [4, 1]
])
print(np.vstack(np.where(a==1)).T)
def cal_edge_length(vertices, v0, v1):
    return np.linalg.norm(vertices[v0] - vertices[v1])
b = np.array([1, 3])
print(np.vstack((a, b)))

In [ ]:
def get_edges_unique_idx(edges_unique, v1, v2):
    e = np.where((edges_unique[:, 0]==v1)&(edges_unique[:, 1]==v2))
    if e[0].shape[0] == 0:
        e = np.where((edges_unique[:, 0]==v2)&(edges_unique[:, 1]==v1))
    return e[0].item()

def get_adj_face(edges_unique, face_adjacency, v1, v2, face_index):
    e = get_edges_unique_idx(edges_unique, v1, v2)
    faces = face_adjacency[e].flatten()
    return faces[faces!=face_index].item()

def get_adj_face_pair(edges_unique, face_adjacency, v1, v2):
    e = get_edges_unique_idx(edges_unique, v1, v2)
    faces = face_adjacency[e].flatten()
    return faces

def can_be_collapsed(vertex_neighbors, v0, v1):
    n0 = vertex_neighbors[v0]
    n1 = vertex_neighbors[v1]
    return np.intersect1d(n0, n1).shape[0] == 2

def cal_edge_length(vertices, v0, v1):
    return np.linalg.norm(vertices[v0] - vertices[v1])

def collapser1(mesh, T, is_fixed):
    
    faces = np.asarray(mesh.faces.copy())

    edges_length = np.asarray(mesh.edges_unique_length.copy())
    edges_unique = np.asarray(mesh.edges_unique.copy())
    edges_need_check = np.vstack(np.where(edges_length < T)).flatten()
    edges_left = edges_need_check.shape[0]
    i = 0

    face_adjacency = np.asarray(mesh.face_adjacency.copy())
    vertex_neighbors = mesh.vertex_neighbors.copy()

    v_is_exist = np.ones((mesh.vertices.shape[0],), dtype=np.bool_)
    f_is_exist = np.ones((mesh.faces.shape[0],), dtype=np.bool_)

    while i < edges_left:
        v0, v1 = edges_unique[edges_need_check[i]]

        # check can be collapsed
        if is_fixed[v0] or is_fixed[v1] or not can_be_collapsed(vertex_neighbors, v0, v1):
            i += 1
            continue

        # reset v0
        mesh.vertices[v0] = 0.5 * (mesh.vertices[v0] + mesh.vertices[v1])
        v_is_exist[v0] = False

        # delete adj faces
        f0, f1 = get_adj_face_pair(edges_unique, face_adjacency, v0, v1)
        f_is_exist[f0] = False
        f_is_exist[f1] = False

        # update topology

            # 1. update face adjacency
        
        f0vs = faces[f0]
        v2 = f0vs[(f0vs!=v0)&(f0vs!=v1)].item()
        v0v2 = get_edges_unique_idx(edges_unique, v0, v2)
        fr = get_adj_face(edges_unique, face_adjacency, v1, v2, f0)
        fa02 = face_adjacency[v0v2]
        fa02[fa02==f0] = fr

        f1vs = faces[f1]
        v3 = f1vs[(f1vs!=v0)&(f1vs!=v1)].item()
        v0v3 = get_edges_unique_idx(edges_unique, v0, v3)
        fr = get_adj_face(edges_unique, face_adjacency, v1, v3, f1)
        fa03 = face_adjacency[v0v3]
        fa03[fa03==f1] = fr
        
        faces[faces==v1] = v0 # change v1 -> v0

            # 2. update edges and their neighbours
        vertex_neighbors[v0].extend(vertex_neighbors[v1])
        vertex_neighbors[v0].remove(v0)
        vertex_neighbors[v0].remove(v1)

        for i, j in np.vstack(np.where(edges_unique==v0)).T:
            nv0 = edges_unique[i, 1 - j] 
            if nv0 != v1:
                el = cal_edge_length(mesh.vertices, *edges_unique[i])
                if el < T:
                    edges_need_check = np.hstack((edges_need_check, i))
                    edges_left += 1

        for i, j in np.vstack(np.where(edges_unique==v1)).T:
            nv1 = edges_unique[i, 1 - j] 
            if nv1 == v0 or nv1 == v2 or nv1 == v3:
                edges_unique[i] = np.array([-1, -1])
            else:
                edges_unique[i, j] = v0
                el = cal_edge_length(mesh.vertices, *edges_unique[i])
                if el < T:
                    edges_need_check = np.hstack((edges_need_check, i))
                    edges_left += 1

        i += 1

    mesh.faces = faces[f_is_exist]
    mesh.remove_unreferenced_vertices()
    
    return v_is_exist

def collapser(mesh, T, is_fixed):
    
    faces = np.asarray(mesh.faces.copy())

    edges_length = np.asarray(mesh.edges_unique_length.copy())
    short_idx = edges_length < T
    edges_unique = np.asarray(mesh.edges_unique.copy())[short_idx]
    face_adjacency = np.asarray(mesh.face_adjacency.copy())[short_idx]
    edges_length = edges_length[short_idx]
    vertex_neighbors = mesh.vertex_neighbors.copy()

    ignore = np.zeros((mesh.vertices.shape[0],), dtype=np.bool_)

    f_is_del = np.ones((mesh.faces.shape[0],), dtype=np.bool_)

    c_count = 0

    collapsed = []

    for edge, adj_faces in zip(edges_unique, face_adjacency):
        v0, v1 = edge

        # check can be collapsed
        if ignore[v0] or ignore[v1] or (is_fixed[v0] and is_fixed[v1]) or not can_be_collapsed(vertex_neighbors, v0, v1):
            continue
        if is_fixed[v1]:
            v0, v1 = v1, v0
            
        # reset v0
        mesh.vertices[v0] = 0.5 * (mesh.vertices[v0] + mesh.vertices[v1])

        # delete adj faces
        f0, f1 = adj_faces
        f_is_del[f0] = False
        f_is_del[f1] = False

        # change v1 -> v0
        faces[faces==v1] = v0

        c_count += 1

        collapsed.append(v1)

        ignore[v1] = True
        for nv in vertex_neighbors[v1]:
            ignore[nv] = True

        ignore[v0] = True
        for nv in vertex_neighbors[v0]:
            ignore[nv] = True
        
    mesh.faces = faces[f_is_del]
    mesh.remove_unreferenced_vertices()
    
    return collapsed

### Split Faces

In [ ]:
# v1v0 onto v1v2
def cal_projection(v0, v1, v2):
    v1v2 = v2 - v1
    if np.linalg.norm(v1v2) < 1e-6:
        return v1
    v1v0 = v0 - v1
    projector = v1v2 / np.linalg.norm(v1v2)
    projectee = v1v0
    t = np.dot(projectee, projector)
    return v1 + t * projector

def replace_vertex(face, v_old, v_new):
    new_face = face.copy()
    new_face[face==v_old] = v_new
    return new_face


#         v0 ---------- v2
#        /        +     /
#      /      vn       /
#    /     +          /   
#  v1 ---------------v3
v_pairs = [[0,1],[1,2],[0,2]]
def splitter(mesh, D, voronoi_poles, is_fixed):

    faces = np.asarray(mesh.faces)
    edges_unique = np.asarray(mesh.edges_unique)
    face_adjacency = np.asarray(mesh.face_adjacency)
    face_angles = np.asarray(mesh.face_angles)
    vertices = np.asarray(mesh.vertices)

    f_is_del = np.zeros((mesh.faces.shape[0],), dtype=np.bool_) # f is deleted or not
    f_is_fixed = np.zeros((mesh.faces.shape[0],), dtype=np.bool_)
    f_to_add = np.asarray(mesh.faces) # all v indexs. Note that here it only adds points and dont remove any.
    v_to_add = np.asarray(mesh.vertices) # positions
    vor_to_add = voronoi_poles.copy()
    
    v_count = mesh.vertices.shape[0]
    v_count_ori = mesh.vertices.shape[0]
    face_count = faces.shape[0]

    
    for face_index in range(face_count):
        face = faces[face_index]
        if not f_is_fixed[face_index] and max(face_angles[face_index]) > D: # 2.1
            
            # get v0,v1,v2
            v0 = face[np.argmax(face_angles[face_index])] # obtuse_vertex
            v1, v2 = face[face!=v0] # v1,v2 are indexes.  
            
            if is_fixed[v1] and is_fixed[v2]:
                continue
            
            # get indexs of adj faces
            face_adj_index = get_adj_face(edges_unique, face_adjacency, v1, v2, face_index)
            face_adj = faces[face_adj_index]

            # TODO: use short_edge
            
            if face_angles[face_adj_index][np.where((face_adj!=v1)&(face_adj!=v2))] < D:
                continue              

            # add vertex vn
            v_count += 1
            v_new = cal_projection(*vertices[[v0, v1, v2]])
            v_to_add = np.vstack((v_to_add, v_new))

            vor_new = cal_projection(*vor_to_add[[v0, v1, v2]])
            vor_to_add = np.vstack((vor_to_add, vor_new))

            # delete face
            f_is_del[face_index]= True
            f_is_del[face_adj_index]= True

            # fix adj faces in this iteration
            for i, j in v_pairs:
                f_is_fixed[get_adj_face(edges_unique, face_adjacency, 
                                        face[i], face[j], 
                                        face_index)] = True
                f_is_fixed[get_adj_face(edges_unique, face_adjacency, 
                                        face_adj[i], face_adj[j], 
                                        face_adj_index)] = True

            # # add faces
            vn = v_count - 1
            f_to_add = np.vstack((f_to_add, replace_vertex(face, v1, vn)))
            f_to_add = np.vstack((f_to_add, replace_vertex(face, v2, vn))) # the order matters! if go v0v2vn, not work
            f_to_add = np.vstack((f_to_add, replace_vertex(face_adj, v1, vn))) # why
            f_to_add = np.vstack((f_to_add, replace_vertex(face_adj, v2, vn)))                
            f_is_del = np.hstack((f_is_del, [False,False,False,False]))

    faces_new = f_to_add[~f_is_del]
    v_add = v_count - mesh.vertices.shape[0]

    mesh.vertices = v_to_add
    mesh.faces = faces_new

    assert mesh.is_watertight

    return v_add != 0, v_add, vor_to_add


### Fix skeleton vertices

In [ ]:
def detect_skeleton_vertices(mesh, T, is_fixed):
    T = T / 10
    vertices = np.asarray(mesh.vertices)
    neighbours = mesh.vertex_neighbors

    for v in range(vertices.shape[0]):
        if is_fixed[v]:
            continue
        badCounter = 0
        for nv in neighbours[v]:
            el = np.linalg.norm(vertices[v] - vertices[nv])
            if el < T and not can_be_collapsed(mesh.vertex_neighbors, v, nv):
                badCounter += 1
        if badCounter >= 2:
            is_fixed[v] = True
    
    return is_fixed

TEST

In [ ]:
def construct_Laplace_Beltrami_matrix(mesh):
    """
    Compute the matrix of un-uniform Laplace-Beltrami operator for a mesh.

    Args:
        mesh:       Trimesh, the data of mesh loaded by Trimesh.
    
    Returns:
        csr_L:      scipy.sparse.csr_matrix, 
                    the sparse representation of the un-uniform Laplace-Beltrami matrix 
    """

    # To contruct csr_matrix
    data = []
    indices = []
    indptr = [0]

    vertices = mesh.vertices
    neighbours = mesh.vertex_neighbors
    for i, n in enumerate(neighbours):
        n_cnt = len(n)
        rdata = np.ones((n_cnt + 1,))
        rdata[n_cnt] = -n_cnt
        rindices = np.zeros((n_cnt + 1,))
        rindices[:n_cnt] = np.array(n)
        rindices[n_cnt] = i
        ri = np.argsort(rindices)
        data.extend(rdata[ri].tolist())
        indices.extend(rindices[ri].tolist())
        indptr.append(indptr[-1] + n_cnt + 1)
        
    csr_L = csr_matrix((data, indices, indptr), shape=(vertices.shape[0], vertices.shape[0]), dtype=np.float32)

    return csr_L

def mesh_reconstruction(mesh, k, is_fixed):

    # Compute sparse Laplace-Beltrami matrix
    csr_L = construct_Laplace_Beltrami_matrix(mesh)
    # csr_L = trimesh_Generate_Tangent_Laplace_matrix(mesh, is_fixed, 0)
    
    # Compute the k-smallest eigenvalues and their eigenvectors
    w, v = eigs(csr_L, k=k, which='SM')
    w = w.real
    v = v.real

    # Get the vertices corrdinates
    vertices = np.asarray(mesh.vertices)

    # To store the corrdinates after reconstruction
    smooth_vertices = np.zeros(vertices.shape)

    # Reconstrction
    ef = vertices.T @ v
    for ch in range(3):
        smooth_vertices[:, ch] = np.sum(np.tile(ef[ch], (vertices.shape[0], 1)) * v, axis=1)
    
    smooth_mesh = trimesh.Trimesh(vertices=smooth_vertices, faces=mesh.faces)
    
    return smooth_mesh

def construct_uniform_Laplace_matrix(mesh, is_fixed):
    """
    Compute the matrix of uniform Laplace-Beltrami operator for a mesh.

    Args:
        mesh:       Trimesh, the data of mesh loaded by Trimesh.
    
    Returns:
        csr_L:      scipy.sparse.csr_matrix, 
                    the sparse representation of the uniform Laplace-Beltrami matrix 
    """

    # To contruct csr_matrix

    print('Constructing Laplace Matrix.')
    
    laplace_dict = {}
    vertices = mesh.vertices
    neighbours = mesh.vertex_neighbors
    for i, n in enumerate(neighbours):
        if is_fixed[i]:
            continue
        nn = len(n)
        laplace_dict[(i, i)] = -nn
        for ni in n:
            laplace_dict[(i, ni)] = 1
    
    # Construct CSR Matrix
    rows, cols = zip(*laplace_dict.keys())
    values = list(laplace_dict.values())
    coo = coo_matrix((values, (rows, cols)), shape=(len(mesh.vertices), len(mesh.vertices)))
    csr = coo.tocsr()

    return csr

### Iteration

In [ ]:
mesh = read_mesh_trimesh('armadillo.obj')
mesh.vertices = mesh_reconstruction(mesh, 300).vertices
write_mesh_o3d('../models/result/recarm.obj', mesh.as_open3d)

In [ ]:
def collapse_Edges1(mesh, T, voronoi_poles, WL, WH, WM, is_fixed, v_color):
    can_collapse = True
    n_collapsed = 0
    while can_collapse:
        v_is_exist = collapser1(mesh, T, is_fixed)
        can_collapse = not np.all(v_is_exist)
        voronoi_poles = voronoi_poles[v_is_exist]
        WL = WL[v_is_exist]
        WH = WH[v_is_exist]
        WM = WM[v_is_exist]
        is_fixed = is_fixed[v_is_exist]
        n_collapsed += np.where(v_is_exist!=True)[0].shape[0]
    print(n_collapsed, "vertices collapsed.")
    assert mesh.is_watertight
    return voronoi_poles, WL, WH, WM, is_fixed

def collapse_Edges(mesh, T, voronoi_poles, WL, WH, WM, is_fixed, v_color):
    can_collapse = True
    n_collapsed = 0
    while can_collapse:
        collapsed = collapser(mesh, T, is_fixed)
        can_collapse = len(collapsed) != 0
        if can_collapse:
            voronoi_poles = np.delete(voronoi_poles, collapsed, axis=0)
            WL = np.delete(WL, collapsed, axis=0)
            WH = np.delete(WH, collapsed, axis=0)
            WM = np.delete(WM, collapsed, axis=0)
            is_fixed = np.delete(is_fixed, collapsed, axis=0)
            v_color = np.delete(v_color, collapsed, axis=0)
        n_collapsed += len(collapsed)
    print(n_collapsed, "vertices collapsed.")
    assert mesh.is_watertight
    return voronoi_poles, WL, WH, WM, is_fixed, v_color

def split_Faces(mesh, D, voronoi_poles, is_fixed):
    n_added = 0
    can_split = True
    t_is_fixed = is_fixed.copy()
    while can_split:
        can_split, v_count, voronoi_poles = splitter(mesh, D, voronoi_poles, t_is_fixed)
        n_added += v_count
        t_is_fixed = np.hstack((is_fixed, np.zeros((n_added,), dtype=bool)))
    print(n_added, "vertices added.")
    return n_added, voronoi_poles

In [ ]:
def cal_diag(vertices):
    return np.max(np.linalg.norm(vertices - vertices[0], axis=1))

it = 1000
wL = 1
wH = 0.1
wM = 0.0

new_mesh = 'armadillo_0.obj'
mesh = read_mesh_trimesh('armadillo.obj')
bbox = mesh.bounding_box
scale = cal_diag(bbox.vertices) * 0.002
print(scale)
assert mesh.is_watertight

voronoi_poles = calculate_voronoi_poles(mesh)


WL = np.ones((mesh.vertices.shape[0])) * wL
WH = np.ones((mesh.vertices.shape[0])) * wH
WM = np.ones((mesh.vertices.shape[0])) * wM

is_fixed = np.zeros((mesh.vertices.shape[0]), dtype=bool)

v_color = np.ones_like(mesh.vertices)

show_lsqr_info = False


for i in range(it):

    laplace_csr = trimesh_Generate_Laplace_matrix(mesh, is_fixed)
    #laplace_csr = trimesh_Generate_Tangent_Laplace_matrix(mesh, is_fixed, scale)
    #laplace_csr = construct_uniform_Laplace_matrix(mesh, is_fixed)

    vertices = np.asarray(mesh.vertices.copy())
    A, b = construct_Equation(diags(WL), diags(WH), diags(WM), vertices, laplace_csr, voronoi_poles)
    sp.io.savemat('../data/Ab%d.mat' % (i + 1), {'A':A, 'b':b}) 
    
    x0 = vertices.copy()
    #x0 = np.zeros_like(vertices)
    
    new_vertices0 = lsqr(A, b[:, 0], damp=0.0, atol=1e-6, btol=1e-6,
                conlim=1e12, iter_lim=None, show=show_lsqr_info, calc_var=False,
                x0=x0[:, 0])
    print(new_vertices0[1], np.linalg.norm(A @ new_vertices0[0] - b[:, 0]))
    new_vertices1 = lsqr(A, b[:, 1], damp=0.0, atol=1e-6, btol=1e-6,
                conlim=1e12, iter_lim=None, show=show_lsqr_info, calc_var=False,
                x0=x0[:, 1])
    print(new_vertices1[1], np.linalg.norm(A @ new_vertices1[0] - b[:, 1]))
    new_vertices2 = lsqr(A, b[:, 2], damp=0.0, atol=1e-6, btol=1e-6,
                conlim=1e12, iter_lim=None, show=show_lsqr_info, calc_var=False,
                x0=x0[:, 2])
    print(new_vertices2[1], np.linalg.norm(A @ new_vertices2[0] - b[:, 2]))

    new_vertices = np.zeros(vertices.shape)
    new_vertices[:, 0] = new_vertices0[0]
    new_vertices[:, 1] = new_vertices1[0]
    new_vertices[:, 2] = new_vertices2[0]

    mesh.vertices = new_vertices

    WL = WL * 2

    write_trimesh_with_color(mesh, '../models/test/' + new_mesh, v_color)
    print('before', mesh.vertices.shape)
    mesh = read_mesh_trimesh('test/' + new_mesh, process=False)
    print('after', mesh.vertices.shape)
    new_mesh = 'armadillo_%d.obj' % (i + 1)

    #wL += 0.01

In [ ]:
new_mesh = 'armadillo_0.obj'
mesh = read_mesh_trimesh('cube.obj', process=False)
assert mesh.is_watertight
is_fixed = np.zeros((mesh.vertices.shape[0]), dtype=bool)
lc = trimesh_Generate_Tangent_Laplace_matrix(mesh, is_fixed, 0)
print(lc)

In [ ]:
l1 = construct_Laplace_Beltrami_matrix(mesh)
l2 = construct_uniform_Laplace_matrix(mesh, is_fixed)
print(np.allclose(l1.toarray(), l2.toarray()))

In [ ]:
lca = lc.toarray()
for n in mesh.vertex_neighbors:
    print(n)
for r in lca:
    print(np.where(r!=0)[0].shape)

In [ ]:
def implicit_Laplacian_mesh_smoothing(mesh, lam=0.01, t=20, is_fixed=None):
    """
    Implicit Laplacian mesh smoothing.

    Args:
        mesh:           Trimesh, the data of mesh loaded by Trimesh.
        lam:            float, represents lambda
        t:              int, iteration round
    
    Returns:
        smooth_mesh:    Trimesh, smoothed mesh.
    """
    
    # Copy the original mesh
    smooth_mesh = mesh.copy()
    
    # Get the vertices corrdinates
    smooth_vertices = np.asarray(mesh.vertices)
    vn = smooth_vertices.shape[0]

    # Compute sparse uniform Laplace-Beltrami matrix
    #csr_uL = construct_uniform_Laplace_matrix(mesh, is_fixed)
    csr_uL = trimesh_Generate_Tangent_Laplace_matrix(mesh, is_fixed, 0)

    # Iterate to smooth the mesh.
    for _ in tqdm(range(t)):
        # Solve the linear system
        smooth_vertices = spsolve(identity(vn) - lam * csr_uL, smooth_vertices)
        smooth_mesh.vertices = smooth_vertices

    return smooth_mesh

In [ ]:
new_mesh = 'armadillo_0.obj'
mesh = read_mesh_trimesh('test/armadillo_20.obj', process=False)
assert mesh.is_watertight
is_fixed = np.zeros((mesh.vertices.shape[0]), dtype=bool)
voronoi_poles = calculate_voronoi_poles(mesh)
vp_cloud = mesh_reconstruction(mesh, 5, is_fixed)
write_mesh_o3d('../models/result/skeleton_rec.obj', vp_cloud.as_open3d)

In [ ]:
mesh = read_mesh_trimesh('test/armadillo_21.obj', process=False)
assert mesh.is_watertight
is_fixed = np.zeros((mesh.vertices.shape[0]), dtype=bool)
smooth_mesh = implicit_Laplacian_mesh_smoothing(mesh, lam=0.01, t=1, is_fixed=is_fixed)
write_mesh_o3d('../models/result/smooth_ske.obj', smooth_mesh.as_open3d)

In [ ]:
def cal_diag(vertices):
    return np.max(np.linalg.norm(vertices - vertices[0], axis=1))

import time
nt = time.strftime("%Y-%m-%d-%H:%M:%S", time.localtime()) 
logfile = open(nt+'.txt', "w")

it = 1000
wL = 1
wH = 10
wM = 20

new_mesh = 'armadillo_0.obj'
mesh = read_mesh_trimesh('armadillo.obj')
bbox = mesh.bounding_box
scale_start = cal_diag(bbox.vertices) * 0.002
scale = cal_diag(bbox.vertices) * 0.002
scale_delta = cal_diag(bbox.vertices) * 0.001
print(it, wL, wH, wM, scale_start, scale, scale_delta, file=logfile)
assert mesh.is_watertight

is_fixed = np.zeros((mesh.vertices.shape[0]), dtype=bool)

#write_mesh_o3d('../models/result/rec' + new_mesh, mesh.as_open3d)
voronoi_poles = calculate_voronoi_poles(mesh_reconstruction(mesh, 200, is_fixed))
#vc_true = read_mesh_trimesh('armadillo/Voronoi.off', process=False)
#voronoi_poles = np.asarray(vc_true.vertices.copy())
#voronoi_poles = calculate_voronoi_poles(mesh_reconstruction(mesh, 300))
vp_cloud = trimesh.Trimesh(vertices=voronoi_poles, faces=mesh.faces)
write_mesh_o3d('../models/result/rec_vpcloud.obj', vp_cloud.as_open3d)


WL = np.ones((mesh.vertices.shape[0])) * wL
WH = np.ones((mesh.vertices.shape[0])) * wH
WM = np.ones((mesh.vertices.shape[0])) * wM


v_color = np.ones_like(mesh.vertices)

show_lsqr_info = False

print(mesh.vertices.shape, voronoi_poles.shape, WL.shape, file=logfile)

voronoi_poles, WL, WH, WM, is_fixed, v_color = collapse_Edges(mesh, scale, voronoi_poles, WL, WH, WM, is_fixed, v_color)

print(mesh.vertices.shape, voronoi_poles.shape, WL.shape, file=logfile)

n_added, voronoi_poles = split_Faces(mesh, (100/180)*np.pi, voronoi_poles, is_fixed)
# voronoi_poles = np.vstack((voronoi_poles, np.zeros((n_added, 3))))
WL = np.hstack((WL, np.ones((n_added,)) * wL))
WH = np.hstack((WH, np.ones((n_added,)) * wH))
WM = np.hstack((WM, np.zeros((n_added,))))
is_fixed = np.hstack((is_fixed, np.zeros((n_added,), dtype=bool)))
for j in range(n_added):
    v_color = np.vstack((v_color, np.array([0., 0., 1.])))

for i in range(it):

    #laplace_csr = trimesh_Generate_Laplace_matrix(mesh, is_fixed)
    laplace_csr = trimesh_Generate_Tangent_Laplace_matrix(mesh, is_fixed, scale)
    #laplace_csr = construct_uniform_Laplace_matrix(mesh, is_fixed)

    vertices = np.asarray(mesh.vertices.copy())
    A, b = construct_Equation(diags(WL), diags(WH), diags(WM), vertices, laplace_csr, voronoi_poles)
    sp.io.savemat('../data/Ab%d.mat' % (i + 1), {'A':A, 'b':b}) 
    
    x0 = vertices.copy()
    #x0 = np.zeros_like(vertices)
    
    new_vertices0 = lsqr(A, b[:, 0], damp=0.0, atol=1e-6, btol=1e-6,
                conlim=1e12, iter_lim=None, show=show_lsqr_info, calc_var=False,
                x0=x0[:, 0])
    print(new_vertices0[1], np.linalg.norm(A @ new_vertices0[0] - b[:, 0]), file=logfile)
    new_vertices1 = lsqr(A, b[:, 1], damp=0.0, atol=1e-6, btol=1e-6,
                conlim=1e12, iter_lim=None, show=show_lsqr_info, calc_var=False,
                x0=x0[:, 1])
    print(new_vertices1[1], np.linalg.norm(A @ new_vertices1[0] - b[:, 1]), file=logfile)
    new_vertices2 = lsqr(A, b[:, 2], damp=0.0, atol=1e-6, btol=1e-6,
                conlim=1e12, iter_lim=None, show=show_lsqr_info, calc_var=False,
                x0=x0[:, 2])
    print(new_vertices2[1], np.linalg.norm(A @ new_vertices2[0] - b[:, 2]), file=logfile)

    new_vertices = np.zeros(vertices.shape)
    new_vertices[:, 0] = new_vertices0[0]
    new_vertices[:, 1] = new_vertices1[0]
    new_vertices[:, 2] = new_vertices2[0]

    mesh.vertices = new_vertices

    mesh = implicit_Laplacian_mesh_smoothing(mesh, lam=0.01, t=1, is_fixed=is_fixed)

    write_trimesh_with_color(mesh, '../models/test/middle_' + new_mesh, v_color)
    
    print(mesh.vertices.shape, voronoi_poles.shape, WL.shape, file=logfile)

    voronoi_poles, WL, WH, WM, is_fixed, v_color = collapse_Edges(mesh, scale, voronoi_poles, WL, WH, WM, is_fixed, v_color)

    print(mesh.vertices.shape, voronoi_poles.shape, WL.shape, file=logfile)

    n_added, voronoi_poles = split_Faces(mesh, (100/180)*np.pi, voronoi_poles, is_fixed)
    # voronoi_poles = np.vstack((voronoi_poles, mesh.vertices[voronoi_poles.shape[0]:]))
    WL = np.hstack((WL, np.ones((n_added,)) * wL))
    WH = np.hstack((WH, np.ones((n_added,)) * wH))
    WM = np.hstack((WM, np.zeros((n_added,))))
    is_fixed = np.hstack((is_fixed, np.zeros((n_added,), dtype=bool)))
    for j in range(n_added):
        v_color = np.vstack((v_color, np.array([0., 0., 1.])))

    is_fixed = detect_skeleton_vertices(mesh, scale_start / 2, is_fixed)
    WL[is_fixed] = 0
    WH[is_fixed] = 0
    WM[is_fixed] = 0
    
    scale += scale_delta / 10
    if scale > 8 * scale_start:
        scale = scale_start

    #wL += 10
    #WL[~is_fixed] = wL

    v_color[is_fixed] = np.array([1., 0., 0.])

    print(mesh.vertices.shape, voronoi_poles.shape, WL.shape, file=logfile)
    #print(mesh.vertex_attributes['vor_idx'].shape, mesh.vertex_attributes['vor_idx'])
    vp_mesh = trimesh.Trimesh(vertices=voronoi_poles, faces=mesh.faces)
    write_mesh_o3d('../models/test/vpcloud_mesh%d.obj' % (i + 1), vp_mesh.as_open3d)

    print(np.where(is_fixed==True)[0].shape, file=logfile)
    write_trimesh_with_color(mesh, '../models/test/' + new_mesh, v_color)
    print('before', mesh.vertices.shape, file=logfile)
    mesh = read_mesh_trimesh('test/' + new_mesh, process=False)
    assert mesh.is_watertight
    print('after', mesh.vertices.shape, file=logfile)
    new_mesh = 'armadillo_%d.obj' % (i + 1)
    print('---------------------------', file=logfile)

# Testing Blocks

In [ ]:
mesh = read_mesh_trimesh('armadillo.obj')

In [ ]:
voronoi_poles = calculate_voronoi_poles(mesh)

In [ ]:
laplace_csr = trimesh_Generate_Laplace_matrix(mesh)

In [ ]:
laplace = laplace_csr.toarray()

In [ ]:
non0idx = np.where(laplace>EPS)
laplace_non0 = laplace[non0idx]
print(np.mean(laplace[non0idx]))

In [ ]:
areas = trimesh_Generate_Area_List(mesh)

In [ ]:
print(np.mean(areas))

In [ ]:
vertices = get_vertices_from_trimesh(mesh)
edges = np.asarray(mesh.edges_unique)
sumlen = 0
for e in edges:
    v0, v1 = vertices[e]
    sumlen += np.linalg.norm(v0 - v1)
print(sumlen/edges.shape[0])

In [ ]:
mesh = read_mesh_trimesh('armadillo.obj')
voronoi_poles = calculate_voronoi_poles(mesh)
laplace_csr = trimesh_Generate_Laplace_matrix(mesh)
wL = 1
wH = 0.1
wM = 0.2
vertices = get_vertices_from_trimesh(mesh)
A, b = construct_Equation(wL, wH, wM, vertices, laplace_csr, voronoi_poles)

new_vertices0 = lsqr(A, b[:, 0])
new_vertices1 = lsqr(A, b[:, 1])
new_vertices2 = lsqr(A, b[:, 2])

new_vertices = np.zeros(vertices.shape)
new_vertices[:, 0] = new_vertices0[0]
new_vertices[:, 1] = new_vertices1[0]
new_vertices[:, 2] = new_vertices2[0]

mesh.vertices = new_vertices
write_mesh_o3d('../models/result/test.obj', mesh.as_open3d)

In [ ]:
name = 'armadillo.obj'
mesh = read_mesh_trimesh(name)
c_count = 0
print(mesh.vertices.shape)
voronoi_poles = calculate_voronoi_poles(mesh)
#voronoi_poles = calculate_voronoi_poles(mesh_reconstruction(mesh, 300))
vp_cloud = trimesh.Trimesh(vertices=voronoi_poles)
write_mesh_o3d('../models/result/rec_vpcloud.obj', vp_cloud.as_open3d)
WL = np.ones((mesh.vertices.shape[0])) * wL
WH = np.ones((mesh.vertices.shape[0])) * wH
WM = np.ones((mesh.vertices.shape[0])) * wM
is_fixed = np.zeros((mesh.vertices.shape[0]), dtype=bool)
voronoi_poles, WL, WH, WM, is_fixed = collapse_Edges1(mesh, 5e-2, voronoi_poles, WL, WH, WM, is_fixed)
print(np.where(f!=True)[0].shape)
print(mesh.is_watertight)
print(mesh.vertices.shape)

In [ ]:
a = np.array([1, 2, 3])
print(np.where((a!=1)&(a!=2)))

In [ ]:
print(mesh.faces.shape[0])
v_count = splitter(mesh, (110/180)*np.pi, 0)
print(v_count)
print(mesh.faces.shape[0])

In [ ]:
mesh = read_mesh_trimesh('armadillo.obj')
print(mesh.vertices.shape[0])
voronoi_poles = calculate_voronoi_poles(mesh)
mesh.vertices = voronoi_poles
write_mesh_o3d('../models/result/vpcloud_mesh.obj', mesh.as_open3d)
print(mesh.is_watertight)

In [ ]:
vc_true = read_mesh_trimesh('armadillo/Voronoi.off', process=False)
print(vc_true.vertices.shape[0])
print(vc_true.is_watertight)
print(np.allclose(vc_true.edges_unique, mesh.edges_unique))

In [ ]:
a = np.linalg.norm(vc_true.vertices - mesh.vertices, axis=1)
print(a.tolist())
idx = np.where(a > 1e-3)[0]
print(idx.shape)
print(idx.tolist())
print(a[idx].tolist())